In [ ]:
import ase
import ase.io
import numpy as np

import matplotlib.pyplot as plt

import tb_mean_field_hubbard as tbmfh

In [ ]:
# Geometry
geom = ase.io.read("geom/clars_goblet.xyz")

# Use the following to scale all CC bonds to a specific length
#tbmfh.utils.scale_to_cc_bond_length(geom, cc_bond=1.42)

# Spin guess is set in atom tags (1 - up, 2 - down)
# one option is to use the following (edit->modify)
# to visualize the spin guess use (view->colors->by tag)
geom.edit()

In [ ]:
# Set up the main calculator object with tight-binding parameters

mfh_model = tbmfh.MeanFieldHubbardModel(
    geom,
    t_list = [2.7, 0.1, 0.4], # List of n'th nearest neighbor hoppings in eV
    charge = 0,
    multiplicity = 1
)

mfh_model.print_parameters()
mfh_model.visualize_spin_guess()

In [ ]:
mfh_model.run_mfh(u = 3.0, print_iter=False, plot=True)

In [ ]:
mfh_model.pp.report(num_orb=2, sts_h=8.0, sts_broad=0.05)

In [ ]:
# Visualize one specific orbital

spin = 0
index = 12
print("Orbital index: %d, spin %d" % (index, spin))
print("Energy: %.6f eV" % mfh_model.evals[spin][index])

# corresponding eigenvector
evec = mfh_model.evecs[spin][index]

# Visualize eigenvector
mfh_model.pp.plot_mo_eigenvector(mo_index=index, spin=spin)

# Visualize the orbital squared
mfh_model.pp.plot_orb_squared_map(plt.gca(), evec)

# Export a map in .itx format

In [ ]:
sts_map, extent = mfh_model.pp.calc_sts_map(energy=1.42, broadening=0.1, h=5.0)

tbmfh.create_itx(data=sts_map, extent=extent, wavename="sts_map", filename="example.itx")

# Series calculations

In [ ]:
geom = ase.io.read("geom/clars_goblet.xyz")

t = 2.7

singlet_mfh_model = tbmfh.MeanFieldHubbardModel(geom, [t], charge=0, multiplicity=1)
triplet_mfh_model = tbmfh.MeanFieldHubbardModel(geom, [t], charge=0, multiplicity=3)
# (for the clar's goblet, the correct triplet state is found also with singlet initial guess)

In [ ]:
# Reproducing Extended Data Fig. 1 from
# Mishra 2020 "Topological frustration induces unconventional magnetism in a nanographene"

u_t_ratios = np.arange(0.5, 1.6, 0.1)

singlet_energies = []
triplet_energies = []

for ut_ratio in u_t_ratios:
    
    u = ut_ratio * t
    
    singlet_mfh_model.run_mfh(u)
    singlet_energies.append(singlet_mfh_model.energy)
    
    triplet_mfh_model.run_mfh(u)
    triplet_energies.append(triplet_mfh_model.energy)
    
singlet_energies = np.array(singlet_energies)
triplet_energies = np.array(triplet_energies)

st_gap = triplet_energies - singlet_energies

In [ ]:
plt.plot(u_t_ratios, st_gap*1000, 'o-')
plt.ylabel("triplet - singlet [meV]")
plt.xlabel("U/t")
plt.show()

# Natural orbitals

In [ ]:
geom = ase.io.read("geom/clars_goblet.xyz")

# "open shell" case, normal MFH
mfh_model = tbmfh.MeanFieldHubbardModel(geom, [2.7, 0.1, 0.4], charge=0, multiplicity=1)
mfh_model.run_mfh(u = 3.0, print_iter=False, plot=False)
mfh_model.pp.calculate_natural_orbitals()

# "closed shell" case (just tight-binding)
tb_model = tbmfh.MeanFieldHubbardModel(geom, [2.7, 0.1, 0.4], charge=0, multiplicity=1)
tb_model.run_mfh(u = 0.0, print_iter=False, plot=False)

In [ ]:
num_orb = 2
h = 8.0

for i_rel in np.arange(num_orb, -num_orb, -1):
    i_mo = int(np.around(0.5 * (mfh_model.num_spin_el[0] + mfh_model.num_spin_el[1]))) + i_rel - 1
    
    fig, axs = plt.subplots(nrows=1, ncols=7, figsize=(7 * mfh_model.figure_size[0], mfh_model.figure_size[1]))
    
    mfh_model.pp.plot_no_eigenvector(i_mo, ax=axs[0])
    mfh_model.pp.plot_orb_squared_map(axs[1], mfh_model.pp.no_evecs[i_mo], h=h)
    
    mfh_model.pp.plot_mo_eigenvector(i_mo, spin=0, ax=axs[2])
    mfh_model.pp.plot_mo_eigenvector(i_mo, spin=1, ax=axs[3])
    mfh_model.pp.plot_sts_map(axs[4], mfh_model.evals[0, i_mo], h=h)
    
    tb_model.pp.plot_mo_eigenvector(i_mo, spin=0, ax=axs[5])
    tb_model.pp.plot_sts_map(axs[6], tb_model.evals[0, i_mo], h=h)
    
    plt.subplots_adjust(wspace=0.0, hspace=0)
    plt.show()